In [42]:
import numpy as np
import pandas as pd
import json, pprint, spacy, string
from pathlib import Path
from copy import deepcopy
from geopy.geocoders import Nominatim
from word2number import w2n
from dateutil import parser
from spacy import *
from spacy.pipeline import *
from spacy.lang.en.stop_words import STOP_WORDS

In [43]:
### Load trained model, data, and psgc
nlp = spacy.load('models')
df = pd.read_json('test_data.json')

In [44]:
def record_to_json(record, f):
    print("Recording to json...")
    if record == 'incident':
        return {
            'disease': str(f[0]),
            'date-start': str(f[1]),
            'date-end': str(f[2]),
            'title': str(f[3]),
            'url': str(f[4]),
            'code': str(f[5]),
            'incident': w2n.word_to_num(str(f[6]))
        }
    elif record == 'state':
        return {
            'disease': str(f[0]),
            'date-start': str(f[1]),
            'date-end': str(f[2]),
            'title': str(f[3]),
            'url': str(f[4]),
            'code': str(f[5]),
            'state': str(f[6])
        }
    elif record == 'change':
        return {
            'disease': str(f[0]),
            'date-start': str(f[1]),
            'date-end': str(f[2]),
            'title': str(f[3]),
            'url': str(f[4]),
            'code': str(f[5]),
            'change': str(f[6])
        }
    return {}

In [45]:
# Returns a list of tuples composed of the name
# and type of the location

def extract_location(text):
    print("Extracting location...")
    locs = []
    # Retrieves the location found in any given sentence
    for word in text:
        if str(word) in '-–—':
            break
        if word not in STOP_WORDS:
            if word.ent_type_ == 'GPE':
                locs.append(str(word))
                
    locs = ' '.join([str(x) for x in locs])
    out = get_centroid(locs)   
    return out

In [46]:
def get_centroid(location):
    print(location)
    print('Generating coordinates...')
    if not location.strip():
        print('No location given.')
        return []
    try:
        print("Attempting to find location...")
        geolocator = Nominatim()
        loc = geolocator.geocode(location)
        print("Attempting to find location...")

        if loc:
            print('Coordinates found.')
            return [loc.longitude, loc.latitude]
        else:
            raise Exception('No coordinates found.')
    except Exception as e:
        print(e)
        return [120.5, 14.5]

In [47]:
"""
features:
    disease
    date
    title
    url
    coordinates
    count
"""
def extract_incident(sent, refs):
    print("Extracting incidents...")
    # Load metadata values
    dis, date, title, url, loc = refs
    date_start = date.strftime('%Y-%m-%d')
    date_end = date.strftime('%Y-%m-%d')
    relations = []
    
    #extracted_loc = extract_location(sent) # Look for location specified in the sentence
        
    # Find all CARDINAL entities, then check if they are 
    # linked to a disease, case, or location
    # If they are, they are probably incidence counts
    if not loc:
        return relations
    for number in filter(lambda w: w.ent_type_ == 'CARDINAL', sent):
        if number.dep_ in ('attr', 'dobj'):
            case = [w for w in number.head.lefts if w.ent_type == 'nsubj']
            if case: 
                count = number.text.replace(',', '')
                relations.append(record_to_json('incident', [dis,  date_start, date_end, title, url, loc, count]))
        else:
            case = number.head.ent_type_
            count = number.text.replace(',', '')
            if case == 'CASE':
                relations.append(record_to_json('incident', [dis,  date_start, date_end, title, url, loc, count]))
            if case == 'LOC':
                relations.append(record_to_json('incident', [dis,  date_start, date_end, title, url, loc, count]))
            
    return relations

In [48]:
"""
features:
    disease
    date
    title
    url
    geocode
    state
"""
def extract_status(sent, refs):
    print("Extracting statuses...")
    # Load metadata values
    dis, date, title, url, loc = refs
    date_start = date.strftime('%Y-%m-%d')
    date_end = date.strftime('%Y-%m-%d')
    relations = []
    
    #extracted_loc = extract_location(sent) # Look for location specified in the sentence
    #if extracted_loc:
    #    loc = search(extracted_loc)
    
    ## I need to find a way to locate a substring of STATE tokens,
    ## concatenate the token.text of each token into one string
    ## then append it into the relations list
    
    ## this one here doesn't implement that yet vvv
    if not loc:
        return relations
    states = filter(lambda x: x.ent_type_ == 'STATE', sent)
    state = ' '.join(map(str, states))
    if state:
        if 'hot' in state:
            state = 'hot'
        elif 'calamity' in state:
            state = 'calamity'
        elif 'outbreak' in state or 'epidemic' in state:
            state = 'outbreak'
        relations.append(record_to_json('state', [dis, date_start, date_end, title, url, loc, state]))
        
    return relations    

In [49]:
"""
features:
    disease
    date
    title
    url
    geocode
    change
"""
def extract_change(sent, refs):
    print("Extracting changes...")
    # Load metadata values
    dis, date, title, url, loc = refs
    date_start = date.strftime('%Y-%m-%d')
    date_end = date.strftime('%Y-%m-%d')
    relations = []
    
    #extracted_loc = extract_location(sent) # Look for location specified in the sentence
    #if extracted_loc:
    #    loc = search(extracted_loc)
    if not loc:
        return relations
    for change in filter(lambda w: w.ent_type_ == 'CHANGE', sent):
        for child in change.children:
            if child.ent_type_ == 'PERCENT':
                if str(change) in ['high, higher', 'increase', 'increasing', 'increased', 'rise', 'rose', 'more']:
                    change = 'rise'
                elif str(change) in ['low', 'lower', 'decrease', 'decreasing', 'decreased', 'fall', 'fell', 'less']:
                    change = 'fall'
                relations.append(record_to_json('change', [dis, date_start, date_end, title, url, loc, change]))
                
    return relations

In [50]:
def extract_refs(article):
    print("Extracting keys from article...")
    dis = article['disease']
    date = article['timestamp'] 
    title = article['title']
    url = article['url']
    sents = [sent.as_doc() for sent in nlp(article['content']).sents]
    loc = extract_location(sents[0])
    return [dis, date, title, url, loc]

In [51]:
def extract(df):
    total_incidents = []
    total_statuses = []
    total_changes = []
    for index, article in df.iterrows():
        doc = nlp(article['content'])
        refs = extract_refs(article)
        deets = [article['title'], article['url']]
        incidents = []
        statuses = []
        changes = []
        for sent in doc.sents:
            i = extract_incident(sent,  refs)
            s = extract_status(sent, refs)
            t = extract_change(sent, refs)
            if i: [incidents.append(x) for x in i]
            if s: [statuses.append(x) for x in s]
            if t: [changes.append(x) for x in t]
        #if incidents: pp.pprint(incidents)
        #if statuses: pp.print(statuses)
        #if changes: pp.print(changes)
    #"""
        if incidents: [total_incidents.append(y) for y in incidents]
        if statuses: [total_statuses.append(y) for y in statuses]
        if changes: [total_changes.append(y) for y in changes]
    
    print("Writing data to JSON files...")
    
    with open('incidents4.json', 'w', encoding='utf-8') as outfile:
        json.dump(total_incidents, outfile, indent=4, ensure_ascii=False)
    with open('statuses4.json', 'w', encoding='utf-8') as outfile:
        json.dump(total_statuses, outfile, indent=4, ensure_ascii=False)
    with open('changes4.json', 'w', encoding='utf-8') as outfile:
        json.dump(total_changes, outfile, indent=4, ensure_ascii=False)
    #"""

In [52]:
extract(df)

Extracting keys from article...
Extracting location...
ISABELA Philippines
Generating coordinates...
Attempting to find location...
Service timed out
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Extracting statuses...
Extracti

Extracting keys from article...
Extracting location...
MANILA Philippines
Generating coordinates...
Attempting to find location...
Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting stat

Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Recording to json...
Recording to json...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes

Extracting location...
BAGUIO CITY Philippines
Generating coordinates...
Attempting to find location...
Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes.

Extracting location...
MANILA Philippines
Generating coordinates...
Attempting to find location...
Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Ext

...

Extracting location...
Summer Capital
Generating coordinates...
Attempting to find location...
Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extractin

Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting keys from article...
Extracting location...
Bayabas Subayon Toledo City
Generating coordinates...
Attempting to find location...
Attempting to find location...
No coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes..

Extracting location...
Camugao , Pinamungajan town
Generating coordinates...
Attempting to find location...
Attempting to find location...
No coordinates found.
Extracting incidents...
Recording to json...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes

Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting keys from article...
Extracting location...
Tuburan
Generating coordinates...
Attempting to find location...
Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Ext

Attempting to find location...
Coordinates found.
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Recording to json...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting keys from article...
Extracting location...
MANILA Philippines
Generating coordinates...
Attempting to find location...
Service timed out
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Extracting incidents...
Recording to json...
Recording to json...
Extracting statuses...
Extracting changes...
Extracting incidents...
Extracting statuses...
Extracting changes...
Recording to json...
Extracting incidents...
Ext

In [54]:
#"""
test = nlp(df['content'][18])
sents = [sent.as_doc() for sent in test.sents]
displacy.render(sents[0], style='ent', jupyter=True)
#"""

In [16]:
"""
a = nlp(df['content'][7])
displacy.render(a, style='ent', jupyter=True)
"""

"\na = nlp(df['content'][7])\ndisplacy.render(a, style='ent', jupyter=True)\n"